In [ ]:
##Function for GA Reporting
def ga_report(property_id, dims, mets, p_strtdate, p_enddate, *args, **kwargs):
    
    filts = kwargs.get('filts', [])
    filt_group = kwargs.get('filt_group', "and_group")
    
###Check if Numpy and Pandas have been imported    
    import sys
    modulenames = [{"np":"numpy"}, {"pd":"pandas"}]
    for modulename in modulenames:
        if ( [*modulename.values()][0] not in sys.modules) :
            print ('You have not imported the {} module as {}'.format([*modulename.values()][0], [*modulename.keys()][0]))
            return

###Exception Handling  
    try:
        #####Check if dimension filters were provided
        if (len(filts)!=0):
            [(print("Dimension filters: ", filt)) for filt in filts]

            try:
                ###FILTER TERMS TO ITERABLE 
                filter_keys_list=[]
                for i in filts:
                    filter_keys_list.append(*i.keys())


                filter_values_list=[]
                for i in filts:
                    filter_values_list.append(*i.values())

                filt_df = pd.DataFrame(data = [filter_keys_list, filter_values_list]).transpose()
                final = []
                for i in range(0,len(filt_df)):
                    final.append(filt_df.loc[i])
            

        ###Check the dimension filter type (OR, NOT, AND). AND is default
                ###OR-FILTER ON DIMENSIONS        
                if (filt_group =="or_group"):
                    dim_filter = FilterExpression(
                                    or_group=FilterExpressionList(
                                        expressions=[
                                            (FilterExpression(
                                                filter=Filter(
                                                    field_name=filt_key,
                                                    string_filter=Filter.StringFilter(value=filt_val),
                                                )
                                            )) for filt_key, filt_val in final]
                                    )
                                )

                ### NOT FILTER ON DIMESIONS
                elif (filt_group =="not_expression"):
                    dim_filter = FilterExpression(
                        not_expression=(FilterExpression(
                        filter=Filter(
                            field_name=[*filts[0].keys()][0],
                            string_filter=Filter.StringFilter(value=[*filts[0].values()][0]),
                        ),

                    )
                                       ) 
                        )

                ### AND FILTER ON DIMENSIONS (DEFAULT)
                else:

                    dim_filter= FilterExpression(
                                    and_group=FilterExpressionList(
                                        expressions=[
                                            (FilterExpression(
                                                filter=Filter(
                                                    field_name=filt_key,
                                                    string_filter=Filter.StringFilter(value=filt_val),
                                                )
                                            )) for filt_key, filt_val in final]
                                    )
                                )
            except Exception as e:
                print("Error: ", e)
                dim_filter = {}
                pass
        else:

            print("No dimension filter")
            dim_filter = {}

        """Runs a simple report on a Google Analytics 4 property."""
        client = BetaAnalyticsDataClient()

        
        ###If dimension filters were provided, run using the dimension filter request 
        if (len(filts)!=0):
        
            request = RunReportRequest(
                property=f"properties/{property_id}",
                dimensions= [Dimension(name=dim_name) for dim_name in dims], 
                metrics= [Metric(name=met_name) for met_name in mets],
                date_ranges=[DateRange(start_date=p_strtdate, end_date=p_enddate)],
                dimension_filter=dim_filter,
                limit=100000,
                #offset=100000,
            )

        ###Else run using the regular request 

        else:
            request = RunReportRequest(
                property=f"properties/{property_id}",
                dimensions= [Dimension(name=dim_name) for dim_name in dims], 
                metrics= [Metric(name=met_name) for met_name in mets],
                date_ranges=[DateRange(start_date=p_strtdate, end_date=p_enddate)],
                limit=100000,
                #offset=100000,
            )
            
        response = client.run_report(request)

        print("Report result: From ", p_strtdate, "to ", p_enddate)
        #for row in response.rows:
        #    print(row.dimension_values[0].value, row.metric_values[0].value) 
        
        ###Output response to DataFrame
        ##Create empty dataframe with dimension and measure columns
        a = np.empty((1,len(dims+mets)))
        a[:] = np.nan
        df = pd.DataFrame(columns = dims+mets, data = a)

        ###Insert data entries into dataframe
        for row in response.rows:
            dim = []
            met = []
            for i in range(0, len(dims)):
                dim.append(row.dimension_values[i].value)
            for j in range(0, len(mets)): 
                met.append(row.metric_values[j].value)
            list = dim+met
            df.loc[len(df)] = list


        df.drop(index = 0,axis = 0, inplace=True)
        return df
    except Exception as e:
        print("Error: ", e)

In [ ]:
###Python Script to Download GA4 Data Using the Google Analytics Data API
###GA4 API will return aggregated metrics on your app/website, given the dimensions you wish to investigate on and the filters to apply
###For more info on configuration, check https://developers.google.com/analytics/devguides/reporting/data/v1/quickstart-client-libraries
###For more info on the dimensions and metrics, check https://ga-dev-tools.google/ga4/dimensions-metrics-explorer/


###Import dependencies
import os
import pandas as pd
import numpy as np
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    Filter,
    FilterExpression,
    FilterExpressionList
)

###Define variables

## Define GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=#"path/to/service/account/service_account.json"

### Define the dimensions, metrics and filters you wish to investigate

##Set the dimensions for the data pull (list)
dimensions = ["isoweek", "year"] #"eventName",

##Set the metrics you wish to explore (list)
metrics = ["newUsers", "activeUsers", "totalUsers"]

##Define filters (list of dictionary)
filters = [] ## Example of filters: [{"eventName": "email_verification_successful"}] # [{"isConversionEvent":"true"}] #[{"platform":"Android"},{"country":"Nigeria"}] ##[{"eventName": "first_open"}, {"platform":"Android"}, {"country":"Nigeria"}]

##Set filter type (either and_group, or_group, not_expression)
filter_group = "and_group" 

##Set the period for which to explore (date in "yyyy-mm-dd" format)
p_strtdate = "2022-06-01"
p_enddate = "today"

##Set the property ID (from the GA project information)
property_id = #"popertyID"


##Call the data into a DataFrame
df = ga_report(property_id, dimensions, metrics, p_strtdate, p_enddate, filts = filters, filt_group = filter_group)

In [ ]:
df.to_csv("new_and_active_users_by_isoweek_Jun_22_date.csv")

In [ ]:
df